In [1]:
import os
os.chdir('..')

In [2]:
from src import CustomGraph, AddData

In [3]:
from src.utils import list_from_file

## documentation

In [4]:
g = CustomGraph()

In [5]:
g.parse("http://www.semanticweb.org/football/", "data/football_antology.rdf")

In [6]:
# add classes, objects etc
file_path = "data/data_lists/"

In [7]:
classes = list_from_file(file_path+"classes")
obj_prop = list_from_file(file_path+"obj_prop")
data_prop = list_from_file(file_path+"data_prop")

In [8]:
for clss in classes:
    g.add_class(clss)
for opr in obj_prop:
    g.add_object_property(opr)
for dpr in data_prop:
    g.add_data_property(dpr)

In [9]:
# fullfill the antology

In [10]:
g.get_list_object_properties()

['competitionOn',
 'hasStatistic',
 'inTeam',
 'isPlacedIn',
 'onDate',
 'onPosition']

In [11]:
AddData.add_stadium(g, "country_name", "city_name", "stadium_name")

Success
